In [1]:
import requests
from bs4 import BeautifulSoup
import traceback
import re

In [ ]:
def getHtmlText(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return

def getStockList(lis, stockUrl):
    html = getHtmlText(stockUrl)
    soup = BeautifulSoup(html, "html.parser")
    a = soup.find_all('a')
    for i in a:
        try:
            href = i.attrs['href']
            lis.append(re.find_all(r"[s][hz]\d{6}", href)[0])
        except:
            continue

def getStockInfo(lis, stockUrl, fpath):
    count = 0
    for stock in lis:
        url = stockUrl + stock + ".html"
        html = getHtmlText(url)
        try:
            if html == "":
                continue
            infoDict = []
            soup = BeautifulSoup(html, "html.parser")
            stockInfo = soup.find('div', attrs={'class':'stock-bets'})

            name = stockInfo.find_all(attrs={'class':'bets-name'})[0]
            infoDict.update({'股票名称':name.text.split()[0]})
            keyList = stockInfo.find_all('dt')
            valueList = stockInfo.find_all('dd')
            for i in range(len(keyList)):
                key = keyList[i].text
                val = valueList[i].text
                infoDict[key] = val
            with open(fpath, 'a', encoding='utf-8') as f:
                f.write(str(infoDict) + '\n')
                count = count + 1
                print('\r当前进度:{:.2f}%'.format(count*100/len(lis)), end='')
        except:
            count = count + 1
            print('\r当前进度:{:.2f}%'.format(count*100/len(lis)), end='')
            traceback.print_exc()
            continue
            
            if __name__ == "__main__":
    stock_list_url = "https://quote.eastmoney.com/stocklist.html"
    stock_info_url = "https://gupiao.baidu.com/stock/"
    output_file = "baiduStockInfo.txt"
    slist = []
    getStockList(slist, stock_list_url)
    getStockInfo(slist, stock_info_url, output_file)